In [435]:
# Read the file from input/
# cd /notebooks/find-political-donors/insight_testsuite/tests/test_1/input/
text_file = open('itcont.txt')
itcont_list = text_file.read().split('\n')

In [436]:
# Given that the data follow strictly the FEC format, for the mungling part, we will combine all the available data
itcont = '|'.join(itcont_list).split('|')
itcont = itcont[:-1]    # Remove the space at the end, the final length of list should be n * 21
if len(itcont) % 21 != 0:
    print('Length error!')
else:
    print('Correct Length!')

Correct Length!


In [437]:
# Visualize the current splitting results
i=0
while (i+1)*21 < len(itcont)+1:
    #print(itcont[i*21: (i+1)*21])
    print(info_extract(itcont[i*21: (i+1)*21]))
    i+=1

None
{'ZIP_CODE': '30004', 'TRANSACTION_AMT': '384', 'TRANSACTION_DT': '01312017', 'OTHER_ID': '', 'CMTE_ID': 'C00177436'}
{'ZIP_CODE': '02895', 'TRANSACTION_AMT': '250', 'TRANSACTION_DT': '01122017', 'OTHER_ID': '', 'CMTE_ID': 'C00384818'}
{'ZIP_CODE': '30750', 'TRANSACTION_AMT': '230', 'TRANSACTION_DT': '01312017', 'OTHER_ID': '', 'CMTE_ID': 'C00177436'}
{'ZIP_CODE': '04105', 'TRANSACTION_AMT': '384', 'TRANSACTION_DT': '01312017', 'OTHER_ID': '', 'CMTE_ID': 'C00177436'}
{'ZIP_CODE': '02895', 'TRANSACTION_AMT': '333', 'TRANSACTION_DT': '01122017', 'OTHER_ID': '', 'CMTE_ID': 'C00384818'}
{'ZIP_CODE': '04105', 'TRANSACTION_AMT': '384', 'TRANSACTION_DT': '01312017', 'OTHER_ID': '', 'CMTE_ID': 'C00177436'}


In [491]:
# Provide several solutions based on potential complication in the data
# Location based: Given the information that the incoming data will always follow the FEC format, we will used | splitter and location
# To prepare the data
def info_extract(FEC_list):
    """
    input: list of strings, representing every single enter from the input file,
    output: length 5 dict
    """
    # Check if length of the list is correct:
    if len(FEC_list)<21:
        return
    
    str_comb = FEC_list[10]+FEC_list[13]
    if (len(FEC_list[15])==0)&(len(FEC_list[0])>0)&(len(FEC_list[13])>0)&(len(FEC_list[14])>0):
        key_info = {}
        key_info['CMTE_ID'] = FEC_list[0]
        key_info['ZIP_CODE'] = FEC_list[10][:5]
        key_info['TRANSACTION_DT'] = FEC_list[13]
        key_info['TRANSACTION_AMT'] = FEC_list[14]
        key_info['OTHER_ID'] = FEC_list[15]
        return key_info
    else:
        return
    
# Dependency function

def round_half(x):
    if x - np.floor(x) < 0.5:
        return int(np.floor(x))
    else:
        return int(np.ceil(x))

In [554]:
# Count sort implementation of the running median
def median_zip(itcont_list):
    input_len = len(itcont_list)
    dict_zip = {}
    output_by_zip = []
    # core_info_track = []
    for i in range(input_len):    # Perform serial processing about the data
        itcont = itcont_list[i].split('|')
        core_info = info_extract(itcont)
        if core_info is None:  # Check if the entry is non-empty
            continue

        # Check if ZIP information is valid
        try:
            len_check = core_info['ZIP_CODE'][4]  # check if 'ZIP_CODE' has at least 5 characters
            zip_code = int(core_info['ZIP_CODE'])  # check if 'ZIP_CODE' contains only digit
            trans_amt = int(core_info['TRANSACTION_AMT'])  # check if 'TRANSACTION_AMT' contains only digit
        except:
            continue

        key = core_info['CMTE_ID'] + core_info['ZIP_CODE']   # Create one dictionanry for each recipient and zip
        if key not in dict_zip:      # First time create the dictionary
            trans_num = 1
            count_arr_below = []    # Store heap as counting sort array for below median
            count_arr_above = []    # Store heap as counting sort array for above median
            count_arr_above_min = 0   # For the above heap, number that represetn lowest entry
            run_median = trans_amt
            ttl_amt = trans_amt
            dict_zip[key] = [count_arr_below, count_arr_above, count_arr_above_min, run_median, trans_num, ttl_amt]  # Create the first entry for the key
            output_by_zip.append(
                '|'.join([core_info['CMTE_ID'], core_info['ZIP_CODE'], str(run_median), str(trans_num), str(ttl_amt)]))
        else:
            trans_num = dict_zip[key][4] + 1
            ttl_amt = dict_zip[key][5]+trans_amt
            if trans_num == 2:
                count_arr_below_max = min(dict_zip[key][3], trans_amt)
                count_arr_below = [0] * (count_arr_below_max+1)  # Given that round is always up,put the small array first
                count_arr_below[count_arr_below_max] +=1
                count_arr_above_min = max(dict_zip[key][3], trans_amt)
                count_arr_above = [0] # Given that round is always up,put the small array first
                count_arr_above[0] +=1
                run_median = round_half((len(count_arr_below)-1+count_arr_above_min)/2)
                ttl_amt = dict_zip[key][5]+trans_amt
                dict_zip[key] = [count_arr_below, count_arr_above, count_arr_above_min, run_median, trans_num, ttl_amt]  # Create the first entry for the key
                output_by_zip.append('|'.join([core_info['CMTE_ID'], core_info['ZIP_CODE'], str(run_median), str(trans_num), str(ttl_amt)]))
            elif trans_num%2 == 1:
                count_arr_below = dict_zip[key][0]
                count_arr_above = dict_zip[key][1]
                count_arr_above_min = dict_zip[key][2]
                run_median = dict_zip[key][3]
                ttl_amt = dict_zip[key][5]+trans_amt
                if (trans_amt>=len(count_arr_below))&(trans_amt<=count_arr_above_min):
                    run_median = trans_amt
                elif trans_amt<len(count_arr_below):
                    run_median = len(count_arr_below)-1
                    count_arr_below[trans_amt]+=1
                    count_arr_below[len(count_arr_below)-1]-=1
                    if count_arr_below[len(count_arr_below)-1] == 0:
                        while count_arr_below[len(count_arr_below)-1]==0:
                            count_arr_below = count_arr_below[:len(count_arr_below)-1]
                elif trans_amt>count_arr_above_min:
                    run_median = count_arr_above_min
                    count_arr_above[0]-=1
                    if trans_amt < count_arr_above_min + len(count_arr_above):
                        min_diff = count_arr_above_min - trans_amt
                        count_arr_above[min_diff] +=1
                    else: 
                        min_diff = trans_amt - (count_arr_above_min + len(count_arr_above)-1)
                        count_arr_above_new = [0] * (len(count_arr_above)+min_diff)
                        count_arr_above_new[:len(count_arr_above)] = count_arr_above
                        count_arr_above_new[len(count_arr_above_new)-1]+=1
                        count_arr_above = count_arr_above_new
                    if count_arr_above[0] == 0:
                        while count_arr_above[0]==0:
                            count_arr_above = count_arr_above[1:len(count_arr_above)]
                            count_arr_above_min+=1
                dict_zip[key] = [count_arr_below, count_arr_above, count_arr_above_min, run_median, trans_num, ttl_amt]  # Create the first entry for the key
                output_by_zip.append('|'.join([core_info['CMTE_ID'], core_info['ZIP_CODE'], str(run_median), str(trans_num), str(ttl_amt)]))  
            elif trans_num%2 == 0:
                count_arr_below = dict_zip[key][0]
                count_arr_above = dict_zip[key][1]
                count_arr_above_min = dict_zip[key][2]
                run_median = dict_zip[key][3]
                ttl_amt = dict_zip[key][5]+trans_amt
                if trans_amt <  run_median:
                    if trans_amt <= len(count_arr_below):
                        count_arr_below[trans_amt-1] +=1
                    else:
                        count_arr_below_max = len(count_arr_below)-1
                        count_arr_below_new = [0] * (trans_amt+1)
                        count_arr_below_new[:count_arr_below_max+1] = count_arr_below
                        count_arr_below = count_arr_below_new
                        count_arr_below[len(count_arr_below)-1]+=1
                    if run_median < count_arr_above_min:
                        min_diff = count_arr_above_min - run_median
                        count_arr_above_new = [0] * (min_diff + len(count_arr_above))
                        count_arr_above_new[min_diff:] = count_arr_above
                        count_arr_above = count_arr_above_new
                        count_arr_above[0]+=1
                        count_arr_above_min = run_median
                    else:
                        count_arr_above[0] +=1
                elif trans_amt >  run_median:
                    if trans_amt < count_arr_above_min:
                        min_diff = count_arr_above_min - trans_amt
                        count_arr_above_new = [0] * (min_diff + len(count_arr_above))
                        count_arr_above_new[min_diff:] = count_arr_above
                        count_arr_above = count_arr_above_new
                        count_arr_above[0]+=1
                        count_arr_above_min = trans_amt
                    elif trans_amt == count_arr_above_min:
                        count_arr_above[0] +=1
                    else:
                        min_diff = trans_amt - (count_arr_above_min + len(count_arr_above)-1)
                        count_arr_above_new = [0] * (len(count_arr_above)+min_diff)
                        count_arr_above_new[:len(count_arr_above)] = count_arr_above
                        count_arr_above_new[len(count_arr_above_new)-1]+=1
                        count_arr_above = count_arr_above_new
                    if run_median == len(count_arr_below)-1:
                        count_arr_below[len(count_arr_below)-1] +=1
                    else:
                        count_arr_below_max = len(count_arr_below)-1
                        count_arr_below_new = [0] * (run_median + 1)
                        count_arr_below_new[:count_arr_below_max+1] = count_arr_below
                        count_arr_below = count_arr_below_new
                        count_arr_below[len(count_arr_below)-1] +=1
                elif trans_amt ==  run_median:
                    if run_median == len(count_arr_below):
                        count_arr_below[trans_amt-1] +=1
                    else:
                        count_arr_below_max = count_arr_below[len(count_arr_below)-1]
                        count_arr_below_new = [0] * (trans_amt+1)
                        count_arr_below_new[:count_arr_below_max+1] = count_arr_below
                        count_arr_below = count_arr_below_new
                        count_arr_below[len(count_arr_below)-1]+=1
                    if run_median ==  count_arr_above_min:
                        count_arr_above[0] +=1
                    else:
                        min_diff = count_arr_above_min - run_median
                        count_arr_above_new = [0] * (min_diff + len(count_arr_above))
                        count_arr_above_new[count_arr_above_min:] = count_arr_above
                        count_arr_above = count_arr_above_new
                        count_arr_above[0]+=1
                        count_arr_above_min = trans_amt
                run_median = round_half((len(count_arr_below)-1+count_arr_above_min)/2)
                dict_zip[key] = [count_arr_below, count_arr_above, count_arr_above_min, run_median, trans_num, ttl_amt]  # Create the first entry for the key
                output_by_zip.append('|'.join([core_info['CMTE_ID'], core_info['ZIP_CODE'], str(run_median), str(trans_num), str(ttl_amt)]))

        #print(trans_amt, count_arr_below, count_arr_above_min, count_arr_above, run_median)
    return output_by_zip

In [555]:
output = median_zip(itcont_list)
output

['C00177436|30004|384|1|384',
 'C00384818|02895|250|1|250',
 'C00177436|30750|230|1|230',
 'C00177436|04105|384|1|384',
 'C00384818|02895|292|2|583',
 'C00177436|04105|384|2|768']

In [489]:
# Heap implementation of the running median
def median_zip(itcont_list):
    input_len = len(itcont_list)
    dict_zip = {}
    output_by_zip = []
    count_arr_max = 0
    count_arr_old = 0
    # core_info_track = []
    for i in range(input_len):
        itcont = itcont_list[i].split('|')
        core_info = info_extract(itcont)
        if core_info is None:  # Check if the entry is non-empty
            continue
            
        # Generate the output for medianvals_by_zip.txt
        # Check if ZIP information is valid
        try:
            len_check = core_info['ZIP_CODE'][4]  # check if 'ZIP_CODE' has at least 5 characters
            zip_code = int(core_info['ZIP_CODE'])  # check if 'ZIP_CODE' contains only digit
            trans_amt = int(core_info['TRANSACTION_AMT'])  # check if 'TRANSACTION_AMT' contains only digit
        except:
            continue

        key = core_info['CMTE_ID'] + core_info['ZIP_CODE']
        if trans_amt > count_arr_max:   # Update the count sort array range
            count_arr_new = [0] * trans_amt
            count_arr_new[0:count_arr_max] = count_arr_old
            count_arr_max = trans_amt
            count_arr_old = count_arr_new
        else:
            count_arr_old[trans_amt]+=1
            
        if key not in dict_zip:
            trans_num = 1
            run_median_max_heap = []
            run_median_min_heap = []
            run_median = trans_amt
            ttl_amt = trans_amt
            flag_odd = 1
            dict_zip[key] = [run_median_max_heap, run_median_min_heap, run_median, trans_num, ttl_amt, flag_odd]  # Create the first entry for the key
            output_by_zip.append(
                '|'.join([core_info['CMTE_ID'], core_info['ZIP_CODE'], str(run_median), str(trans_num), str(ttl_amt)]))
        else:
            trans_num = dict_zip[key][2] + 1
            flag_odd = 1-dict_zip[key][6]
            run_median_max_heap  = dict_zip[key][0]
            run_median_min_heap  = dict_zip[key][1]
            run_median = dict_zip[key][2]
            if ~flag_odd:     # From odd to even sequence
                if trans_amt > run_median:
                    run_median_max_heap.append(run_median)
                    if trans_amt < run_median_min_heap:
                        run_median_min_heap = trans_amt
                    elif trans_amt > run_median_min_heap:
                        
                elif trans_amt < run_median:
                    run_median_min_heap = run_median
                    if trans_amt > run_median_max_heap:
                        run_median_max_heap = trans_amt
                run_median = round_half((run_median_max_heap+run_median_min_heap)/2)
            if flag_odd:     # From even to odd sequence
                if (trans_amt > run_median_max_heap)&(trans_amt < run_median_min_heap):
                    run_median = trans_amt
                elif (trans_amt < run_median_max_heap):
                    
                else:
                    run_median_min_heap = run_median
                    if trans_amt > run_median_max_heap:
                        run_median_max_heap = trans_amt
                run_median = round_half((run_median_max_heap+run_median_min_heap)/2)
                
            if trans_num == 2:
                run_median = round_half(sum(run_median_cache) / 2)
            elif trans_num % 2 == 0:
                run_median_cache = run_median_cache[1:2]
                run_median = round_half(sum(run_median_cache) / 2)
            else:
                run_median = run_median_cache[1]
            ttl_amt = dict_zip[key][3] + trans_amt
            dict_zip[key] = [run_median_cache, run_median, trans_num, ttl_amt]  # Update the entry for the key
            output_by_zip.append(
                '|'.join([core_info['CMTE_ID'], core_info['ZIP_CODE'], str(run_median), str(trans_num), str(ttl_amt)]))

        # Generate the output for medianvals_by_date.txt
        try:
            len_check = core_info['ZIP_CODE'][4]  # check if 'ZIP_CODE' has at least 5 characters
            zip_code = int(core_info['ZIP_CODE'][0:4])  # check if 'ZIP_CODE' has the correct format
            trans_amt = int(core_info['TRANSACTION_AMT'])  # check if 'TRANSACTION_AMT' has the correct format
        except:
            continue

    return output_by_zip

In [556]:
def date_format(date_str, amt_str):
    """
        date_str: string to check
    """
    try:
        month = int(date_str[:2])
        date = int(date_str[2:4])
        yr = int(date_str[-4:])
        amt = int(amt_str)
        
        if (0<month<12)&(0<date<32)&(1700<yr<2018):
            return True
        else:
            return False
    except:
        return False

In [557]:
# Define the function to calculate the median by recipient and date
# Find unique combination of CMTE_ID and date

# Combine all the strings
def median_date(itcont_list):
    
    itcont = '|'.join(itcont_list).split('|')
    itcont = itcont[:-1]    # Remove the space at the end, the final length of list should be n * 21

    # Obtain the index with valid data info
    num_entry = int(len(itcont)/21)

    # Find the entries with Other_Id as [] and TRANS_AMT, TRANS_DT of correct format:

    key_id = [i for i in range(num_entry) if (itcont[i*21+15]=='')&(date_format(itcont[i*21+13], itcont[i*21+14]))&(len(itcont[i*21])>0)]

    # Generate combitation keys with CMTE_ID and TRANS_DT
    keys = [itcont[i*21]+itcont[i*21+13] for i in key_id]
    unique_key = list(set(keys))
    unique_key.sort()
    output_by_date = []
    for j in range(len(unqiue_key)):
        indices = [key_id[i] for i,x in enumerate(keys) if x == unqiue_key[j]]
        trans_num = len(indices)
        trans_amt = [int(itcont[i*21+14]) for i in indices]
        trans_amt.sort()
        ttl_trans = sum(trans_amt)
        if len(trans_amt) == 1:
            median_trans= trans_amt
        elif len(trans_amt) == 2:
            median_trans= round(ttl_trans/2)
        elif len(trans_amt)%2 == 0:
            mid_idx = int(len(trans_amt)/2-1)
            median_trans = round(sum(trans_amt[mid_idx:mid_idx+2])/2)
        else:
            mid_idx = int(len(trans_amt)/2)+1
            median_trans = round(trans_amt[mid_idx])
        len_key = len(unqiue_key[j])
        output_by_date.append('|'.join([unqiue_key[j][:len_key-8], unqiue_key[j][len_key-8:], str(median_trans),str(trans_num),str(ttl_trans)]))
    
    return output_by_date

In [558]:
output_by_date = median_date(itcont_list)
output_by_date

['C00177436|01312017|384|4|1382', 'C00384818|01122017|292|2|583']